In [1]:
!pip install torch
!pip install datasets
!pip install transformers
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import tkinter as tk
from tkinter import messagebox
import torch.nn.functional as F

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import tkinter as tk
from tkinter import messagebox
import torch.nn.functional as F
# Model Training and Saving
# Load the dataset from Hugging Face
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment", cache_dir="./CentralCache")
# Print dataset information
print(f"Dataset Info:")
print(f"Train dataset length: {len(dataset['train'])}")
print(f"Validation dataset length: {len(dataset['validation'])}")
print(f"Test dataset length: {len(dataset['test']) if 'test' in dataset else 'No test data available'}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/859k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

Dataset Info:
Train dataset length: 9543
Validation dataset length: 2388
Test dataset length: No test data available


In [7]:
# Load the tokenizer and model (DistilBERT-based model)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./CentralCache")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, cache_dir="./CentralCache")
# Function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)
# Tokenize datasets and prepare them for training
train_dataset = dataset['train'].map(tokenize_function, batched=True)
val_dataset = dataset['validation'].map(tokenize_function, batched=True)
# Rename the label column and remove unnecessary columns
train_dataset = train_dataset.rename_column("label", "labels").remove_columns(["text"])
val_dataset = val_dataset.rename_column("label", "labels").remove_columns(["text"])

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [8]:
import torch

# Define a compute_metrics function for evaluation
def compute_metrics(p):
    # Convert predictions to a tensor if they are numpy arrays
    preds = torch.tensor(p.predictions) if not isinstance(p.predictions, torch.Tensor) else p.predictions
    preds = torch.argmax(preds, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    accuracy = accuracy_score(p.label_ids, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [9]:
# Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory for model checkpoints
    evaluation_strategy="epoch",     # Evaluation after every epoch
    learning_rate=7e-5,              # Learning rate
    per_device_train_batch_size=16,   # Batch size per device during training
    per_device_eval_batch_size=16,   # Batch size per device during evaluation
    num_train_epochs=4,              # Number of training epochs
    weight_decay=0.02,               # Strength of weight decay
    logging_dir="./logs",            # Directory for logs
    logging_steps=10,                # Log every 10 steps
    save_total_limit=2,              # Limit the number of checkpoints saved
)
# Create the Trainer instance
trainer = Trainer(
    model=model,                       # The model to be trained
    args=training_args,                # Training arguments
    train_dataset=train_dataset,       # Training dataset
    eval_dataset=val_dataset,          # Evaluation dataset
    compute_metrics=compute_metrics,   # Compute metrics for evaluation
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.406400,0.362696,0.866415,0.870955,0.866415,0.867660
2,0.177100,0.386238,0.878141,0.878143,0.878141,0.877926
3,0.203500,0.515091,0.882328,0.881973,0.882328,0.881603
4,0.104300,0.624709,0.889028,0.888301,0.889028,0.888597


TrainOutput(global_step=2388, training_loss=0.2262652777821774, metrics={'train_runtime': 1916.4964, 'train_samples_per_second': 19.918, 'train_steps_per_second': 1.246, 'total_flos': 5056635717808128.0, 'train_loss': 0.2262652777821774, 'epoch': 4.0})

In [11]:

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")


Evaluation Results: {'eval_loss': 0.6247093677520752, 'eval_accuracy': 0.8890284757118928, 'eval_precision': 0.8883014258353117, 'eval_recall': 0.8890284757118928, 'eval_f1': 0.8885967694294593, 'eval_runtime': 38.3321, 'eval_samples_per_second': 62.298, 'eval_steps_per_second': 3.913, 'epoch': 4.0}


In [16]:
import shutil
import os
from google.colab import files
# Save the model and tokenizer
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

# Compress the saved model directory into a zip file
shutil.make_archive("/content/sentiment_model", 'zip', './sentiment_model')

# To download the zip file
files.download("/content/sentiment_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>